In [45]:
import numpy as np

In [46]:
data = np.load("results.npy")
labels = data.files
labels

['timestamps', 'states', 'controls']

In [47]:
timestamps = data['timestamps']
states = data['states'][0]
controls = data['controls'][0]

print(timestamps.shape, states.shape, controls.shape)

(1, 100352) (16, 100352) (12, 100352)


In [48]:
data_table = np.vstack((timestamps, states))[0:4].T
data_table.shape

(100352, 4)